In [6]:
# おまじない
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from IPython.display import display
from tensorflow import keras

# 各列の名称
column_names = ["MPG", "Cylinders", "Displacement", "Horsepower", "Weight", "Acceleration", "Model Year", "Origin", "Car Names"]

# データの整理
dataset = pd.read_excel("drive/MyDrive/auto-mpg.xlsx", sheet_name=0, names=column_names)    # データの読み込み
dataset = dataset.drop("Car Names", axis=1)     # 車名の列を削除
dataset["Origin"] = dataset["Origin"].map({1: "USA", 2: "Europe", 3: "Japan"})  # 1をUSA、2をEurope、3をJapanに置換
dataset = pd.get_dummies(dataset, columns=["Origin"])   # カテゴリデータを数値データに変換

display(dataset)    # 表の表示
display(dataset.describe())  # 統計情報の表示

# データを学習用とテスト用に分離
train_dataset = dataset.sample(frac=0.8, random_state=0)    # 8割を学習用に
test_dataset = dataset.drop(train_dataset.index)            # 残りをテスト用に

# 説明変数と目的変数に分離
train_x = train_dataset.copy()  # train_xにtrain_datasetをコピー
test_x = test_dataset.copy()    # test_xにtest_datasetをコピー

train_y = train_x.pop("MPG")    # train_xからMPGの列を取り出しtrain_yに代入
test_y = test_x.pop("MPG")      # test_xからMPGの列を取り出しtest_yに代入

# np.array形式に変換
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

print(train_x)
print(train_y)

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin_Europe,Origin_Japan,Origin_USA
0,18.0,8,307.0,130,3504,12.0,70,0,0,1
1,15.0,8,350.0,165,3693,11.5,70,0,0,1
2,18.0,8,318.0,150,3436,11.0,70,0,0,1
3,16.0,8,304.0,150,3433,12.0,70,0,0,1
4,17.0,8,302.0,140,3449,10.5,70,0,0,1
...,...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,82,0,0,1
388,44.0,4,97.0,52,2130,24.6,82,1,0,0
389,32.0,4,135.0,84,2295,11.6,82,0,0,1
390,28.0,4,120.0,79,2625,18.6,82,0,0,1


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin_Europe,Origin_Japan,Origin_USA
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,0.173469,0.201531,0.625000
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.379136,0.401656,0.484742
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,0.000000,0.000000,0.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,0.000000,0.000000,0.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,0.000000,0.000000,1.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,0.000000,0.000000,1.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,1.000000,1.000000,1.000000


[[  4.  90.  75. ...   0.   0.   1.]
 [  4. 140.  88. ...   0.   0.   1.]
 [  8. 350. 160. ...   0.   0.   1.]
 ...
 [  4. 108.  93. ...   0.   1.   0.]
 [  4.  83.  61. ...   0.   1.   0.]
 [  4. 107.  86. ...   1.   0.   0.]]
[28.  22.3 12.  38.  33.8 19.4 38.1 30.  20.  20.  27.  16.5 24.5 11.
 16.9 33.7 21.6 14.  26.  28.4 13.  16.  20.  25.  41.5 14.  25.8 25.1
 20.  17.  20.  31.6 22.  26.  21.  29.8 31.  13.  16.  14.  15.  44.6
 31.3 16.  29.  16.  29.  13.  17.5 18.  26.  15.  10.  22.  34.3 30.7
 20.2 22.  33.  21.  22.  24.  31.5 15.  26.  16.  14.  27.  25.  40.8
 36.1 30.  17.6 15.5 23.  14.  26.  19.2 31.5 33.5 20.5 34.2 24.  24.
 14.  23.9 24.  32.9 31.8 21.5 25.5 15.  21.5 19.  38.  23.  35.1 23.
 31.  39.4 12.  25.  24.  26.5 34.7 28.8 28.  18.2 44.  14.  15.5 36.
 25.5 19.2 26.  25.  27.  17.5 31.  17.  26.  32.3 11.  20.5 13.  15.
 26.  33.5 39.  32.2 21.  15.  23.  28.  15.  26.4 12.  24.2 15.5 23.
 29.  27.  15.  29.  21.  27.4 14.  23.2 15.  15.  35.7 16.2 18.  16